In [10]:
import re
import inspect

In [25]:
def newSplit(sql_str, dl):    
    in_bracket = 0        #indicates if current caracter is within circular bracket
    in_quote = 0          #indicates if current caracter is within quote
    split_cache = ''      #current split element cache
    split_arr = []        #result array

    for ch in sql_str:
        if ch == '(' :      #update in_bracket value if '(' not in quote
            if in_quote == 0:
                in_bracket = in_bracket + 1
            split_cache = split_cache + ch #add character into split cache
        elif ch == ')':     #update in_bracket value if ')' not in quote
            if in_quote == 0:
                in_bracket = in_bracket - 1
            #add character into split cache
            split_cache = split_cache + ch
        #update in_quote value based on whether single quote found before
        elif ch == "'":
            if in_quote == 0:
                in_quote = in_quote + 1
            else:
                in_quote = in_quote - 1
            #add character into split cache
            split_cache = split_cache + ch
        #if given delimeter found
        elif ch == dl:
            #if delimeter not in circular bracket or single quote
            if in_bracket == 0 and in_quote == 0:
                #move split cache content to split array
                split_arr.append(split_cache)
                #reset split cache
                split_cache = ''
            else:
                #add character into split cache
                split_cache = split_cache + ch
        else:
            #add character into split cache
            split_cache = split_cache + ch
    #add remaining characters in output list
    split_arr.append(split_cache)
    return split_arr  

In [26]:
def newLower(in_str):
    qt_str=[]
    #make list substrings with in quotes
    qt_str=re.findall(r'\'(.*?)\'',in_str)
    #replace the quoted substrings with special string
    new_str = re.sub(r'\'(.*?)\'', r'>-o-<', in_str)
    #apply lowercase function on replaced string
    new_str = new_str.lower()
    #replace spaecial string with corresponding quoted substring
    for element in qt_str:
        new_str = re.sub(r'>-o-<', "'" + element + "'", new_str, 1)
    return new_str


In [30]:
def lineno():
    """Returns the current line number in our program."""
    return inspect.currentframe().f_back.f_lineno

In [112]:
        ins_sel = re.match(r'^\bINSERT\s+(?:INTO\s+)?([\w\[\]#\.]+).*\bSELECT\s+.*\bFROM', stmt, re.S|re.I)
        ins_val = re.match(r'^\bINSERT\s+(?:INTO\s+)?([\w\[\]#\.]+).*?\bVALUES\s*\(', stmt, re.S|re.I)
        if ins_sel:         
            try:
                dest_tbl = ins_sel.group(1).strip()
                #cnv_ds.cntx.logger.add_log('INFO', f'Converting INSERT INTO statement by SELECT. Inserting into {dest_tbl}')
                if dest_tbl:
                    #get table df name
                    table_df = dest_tbl+'_df'
                
                #Check if Select is present in Source of the Insert statement 
                src_select = re.search(dest_tbl + r'(.*)[\(]?\s*\bSELECT\s+(.)*\bFROM\s+((\w*)[\.]?(\w*)[\.]?(\w*))', stmt, re.S|re.I)
                print(lineno())
                if src_select:
                    #Check if no column name is specified , only Select * is present
                    print(lineno())
                    select_star = re.search(dest_tbl + r'\s*[\(]?\s*\bSELECT\s+[\*\s]?\s*\bFROM\s+((\w*)[\.]?(\w*)[\.]?(\w*))', stmt, re.S|re.I)
                    if select_star:
                        #get the select * string
                        print(lineno())
                        select_str = re.split("SELECT",stmt,1, re.I)     
                        if select_str.strip()[-1] == ';':
                            select_str = select_str.strip()[:-1]
                        select_str = ('select' + (select_str[1])).strip().strip(')')
                        print(lineno())
                    else:
                        print(lineno())
                        #when column names are specified in the select list
                        if re.search(r'(.)\s*SELECT\b', stmt, re.S|re.I).group(1) == '(':
                        #replace starting bracket with special string
                            stmt = re.sub(r'\((\s*)(SELECT\b)', r'‹\1\2', stmt, 1, flags=re.S|re.I)
                        print(lineno())
                        #replace FROM keyword with special character 
                        stmt = re.sub(r"\bFROM\b", 'ƒ', stmt, flags=re.S|re.I)
                        print(lineno())
                        #custom split insert statement 
                        stmt_part = newSplit(stmt, 'ƒ')
                        print(lineno())
                        #Keeping the first part of stmt_part[0] and appending rest of the array strings in stmt_part[1]
                        for i in range(2,len(stmt_part)):
                             stmt_part[1]=stmt_part[1]+' ƒ '+stmt_part[i]
                        print(lineno())
                    
                        #Replacing special character to from in first part of the split
                        stmt_part_1 = re.sub(r'ƒ', 'from', re.sub(r'‹', r'(', stmt_part[0], flags=re.S|re.I), flags=re.S|re.I)
                        #Replacing special character to from in second part of the split
                        stmt_part_2 = re.sub(r'ƒ', 'from', re.sub(r'‹', r'(', stmt_part[1], flags=re.S|re.I), flags=re.S|re.I)
                        print(lineno())
                        #get Destination table column list
                        col_list_str = re.search(dest_tbl + r'\s*\((.*?)\)[\(\s]*\bSELECT\b', stmt_part_1, re.S|re.I).group(1)
                        col_list = newSplit(col_list_str, ',')
                        print(lineno())
                        #get insert values from the select list
                        val_list_str = re.search(r'\bSELECT\b\s+(.*)', stmt_part_1, re.S|re.I).group(1)
                        val_list = newSplit(val_list_str, ',')
                        print(lineno())
                        cnct_str = ''
                        for i in range(len(col_list)):
                            col = col_list[i].strip()
                            val = val_list[i].strip()

                            #remove alias from column value
                            if re.match(r'^\bCASE\b', val, re.S|re.I):
                                #ignore END keyword as alias for column value with CASE statement
                                val = re.sub(r'(?<=END)\s*(AS)?[ \t]+\w+$', '', val, flags=re.S|re.I)
                            else:
                                val = re.sub(r'(\s*AS)?[ \t]+[a-zA-Z]\w+$', '', val, flags=re.S|re.I)
                                
                            cnct_str += val + ' as ' + col + ',\n'                                
                            
                        cnct_str = cnct_str[:-2]
                        select_str = 'select ' + cnct_str + "\nfrom " + stmt_part_2
                    
                    #table to dataframe and replace variables
                    select_str = util.replaceTableWithDF(cnv_ds, select_str)
                    select_str = self.replaceVariables(cnv_ds, select_str)
                    
                    table_df_tmp = table_df + '_1'
                    #pyspark code to remove deleted records from data frame
                    cnv_code += f"{table_df_tmp} = spark.sql({select_str})\n"
                    cnv_code += f"mod_df['{table_df}'] = mod_df['{table_df}'].union({table_df_tmp})\n"
                    cnv_code += f"mod_df['{table_df}'].createOrReplaceTempView('{table_df}')\n" 
                    cnv_code += f"rowcount_df = {table_df_tmp}\n\n"                    
                    print(lineno())
            except Exception as e:
                #cnv_ds.cntx.logger.add_log('ERROR','Error occurred while converting below INSERT statement,')
                #cnv_ds.cntx.logger.add_log_details(self.original_text)  
                #cnv_ds.cntx.logger.add_log_details(str(e))
                print('Error occurred while converting below INSERT statement',lineno())
                cnv_code = ''
        
        if ins_val:
            try:
                #get tablename, column and value list
                match_grp = re.match(r'INSERT\s+(?:INTO\s+)?([\w\[\]#\.]+)\s+\(?(.*?)\)?\s*VALUES\s*\((.*?)\);', stmt, re.S|re.I)
                table = match_grp.group(1).strip()
                cols = match_grp.group(2).strip()
                vals = match_grp.group(3).strip()
                
                print(f'Converting INSERT INTO statement by VALUE. Inserting into {table}')
                print(inspect.currentframe().f_back.f_lineno)
                print('\n')
                #get table df name
                table_df = cnv_ds.table_df_map[table]
    
                #create insert column list
                if cols:
                    cols = re.sub(r'\s', '', cols, flags=re.S|re.I)
                    col_list = newSplit(cols, ',')
                else:
                    #cnv_ds.cntx.logger.add_log('ERROR','Can not convert insert statement if column list not specified')
                    print('Can not convert insert statement if column list not specified',lineno())
    
                #create insert value list
                vals = re.sub(r'[\r\n]+', '', vals, flags=re.S|re.I) 
                val_list = newSplit(vals, ',')
    
                #create select statement
                select_str = 'select '
                for i in range(len(col_list)):
                    select_str += val_list[i] + ' as ' + col_list[i] + ',\n'
                select_str = select_str[:-2]
                
                #replace db table names with corresponding data frame name
                #select_str = util.replaceTableWithDF(cnv_ds, select_str)
                #select_str = self.replaceVariables(cnv_ds, select_str)
                    
                table_df_tmp = table_df + '_1'
                #pyspark code to remove deleted records from data frame
                cnv_code += f"{table_df_tmp} = spark.sql({select_str})\n"
                cnv_code += f"mod_df['{table_df}'] = mod_df['{table_df}'].union({table_df_tmp})\n"
                cnv_code += f"mod_df['{table_df}'].createOrReplaceTempView('{table_df}')\n" 
                cnv_code += f"rowcount_df = {table_df_tmp}\n\n"

            except Exception as e:
                print('Error occurred while converting below INSERT statement',lineno())
                #cnv_ds.cntx.logger.add_log_details(self.original_text)
                #cnv_ds.cntx.logger.add_log_details(str(e))                
                cnv_code = ''
            
        print(cnv_code)    

13
16
27
32
35
38
42
48
Error occurred while converting below INSERT statement 89



In [111]:
stmt='''INSERT INTO [LOOKUP].[MASTERCONTRACT] (
                [Key],
                [Changed],
                [CIS_ID],
                LOADCONTROLCREATEID,
                LOADCONTROLUPDATEID)
            SELECT  DISTINCT 
                '<Key>'
                    + '<DEBTORNUM>'+CONVERT(VARCHAR(20),A.DEBTORNUM)+'</DEBTORNUM>'
                    +'<PREMNUM>'+CONVERT(VARCHAR(20),A.PREMNUM)+'</PREMNUM>'
                    +'<SUPPLYREQDATE>'+CONVERT(VARCHAR(20),A.SUPPLYREQDATE)+'</SUPPLYREQDATE>'
                +'</Key>',
                1 AS Changed,
                1 AS CIS_ID,
                A.LOADCONTROLCREATEID,
                A.LOADCONTROLCREATEID
            FROM    ICMSTAGE_PEACETX.PM_E_CONSPREM_TB A
            LEFT    JOIN ICMTRANS_PEACETX.vw_MasterContractLookup B
                ON  A.[DEBTORNUM] = B.[DEBTORNUM]
                AND A.[PREMNUM] = B.[PREMNUM]
                AND A.SUPPLYREQDATE = B.SUPPLYREQDATE 
            WHERE   B.[DEBTORNUM]       IS NULL;'''

In [63]:
src_select = re.search(dest_tbl + r'(.*)[\(]?\s*\bSELECT\s+(.)*\bFROM\s+((\w*)[\.]?(\w*)[\.]?(\w*))', stmt, re.S|re.I)
src_select

In [81]:
        ins_sel = re.match(r'^\bINSERT\s+(?:INTO\s+)?([\w\[\]#\.]+).*\bSELECT\s+.*\bFROM', stmt, re.S|re.I)
        ins_val = re.match(r'^\bINSERT\s+(?:INTO\s+)?([\w\[\]#\.]+).*?\bVALUES\s*\(', stmt, re.S|re.I)

In [86]:
print(ins_val)

None


In [87]:
print(ins_sel)

<re.Match object; span=(0, 679), match="INSERT INTO [LOOKUP].[MASTERCONTRACT] (\n        >


In [89]:
                dest_tbl = ins_sel.group(1).strip()
                #cnv_ds.cntx.logger.add_log('INFO', f'Converting INSERT INTO statement by SELECT. Inserting into {dest_tbl}')
                dest_tbl

'[LOOKUP].[MASTERCONTRACT]'

In [90]:
                    #get table df name
                    table_df = dest_tbl+'_df'           

In [93]:
#Check if Select is present in Source of the Insert statement 
src_select = re.search(dest_tbl + r'(.*)[\(]?\s*\bSELECT\s+(.)*\bFROM\s+((\w*)[\.]?(\w*)[\.]?(\w*))', stmt, re.S|re.I)
src_select

<re.Match object; span=(13, 716), match="LOOKUP].[MASTERCONTRACT] (\n                [Key]>

In [94]:
select_star = re.search(dest_tbl + r'\s*[\(]?\s*\bSELECT\s+[\*\s]?\s*\bFROM\s+((\w*)[\.]?(\w*)[\.]?(\w*))', stmt, re.S|re.I) 
select_star  

In [102]:
                        if re.search(r'(.)\s*SELECT\b', stmt, re.S|re.I).group(1) == '(':
                        #replace starting bracket with special string
                            stmt = re.sub(r'\((\s*)(SELECT\b)', r'‹\1\2', stmt, 1, flags=re.S|re.I)

In [103]:
stmt

"INSERT INTO [LOOKUP].[MASTERCONTRACT] (\n                [Key],\n                [Changed],\n                [CIS_ID],\n                LOADCONTROLCREATEID,\n                LOADCONTROLUPDATEID)\n            SELECT  DISTINCT \n                '<Key>'\n                    + '<DEBTORNUM>'+CONVERT(VARCHAR(20),A.DEBTORNUM)+'</DEBTORNUM>'\n                    +'<PREMNUM>'+CONVERT(VARCHAR(20),A.PREMNUM)+'</PREMNUM>'\n                    +'<SUPPLYREQDATE>'+CONVERT(VARCHAR(20),A.SUPPLYREQDATE)+'</SUPPLYREQDATE>'\n                +'</Key>',\n                1 AS Changed,\n                1 AS CIS_ID,\n                A.LOADCONTROLCREATEID,\n                A.LOADCONTROLCREATEID\n            ƒ    ICMSTAGE_PEACETX.PM_E_CONSPREM_TB A\n            LEFT    JOIN ICMTRANS_PEACETX.vw_MasterContractLookup B\n                ON  A.[DEBTORNUM] = B.[DEBTORNUM]\n                AND A.[PREMNUM] = B.[PREMNUM]\n                AND A.SUPPLYREQDATE = B.SUPPLYREQDATE \n            WHERE   B.[DEBTORNUM]       IS 

In [107]:
                        stmt = re.sub(r"\bFROM\b", 'ƒ', stmt, flags=re.S|re.I)
                        stmt
                        stmt_part = newSplit(stmt, 'ƒ')
                        stmt_part

["INSERT INTO [LOOKUP].[MASTERCONTRACT] (\n                [Key],\n                [Changed],\n                [CIS_ID],\n                LOADCONTROLCREATEID,\n                LOADCONTROLUPDATEID)\n            SELECT  DISTINCT \n                '<Key>'\n                    + '<DEBTORNUM>'+CONVERT(VARCHAR(20),A.DEBTORNUM)+'</DEBTORNUM>'\n                    +'<PREMNUM>'+CONVERT(VARCHAR(20),A.PREMNUM)+'</PREMNUM>'\n                    +'<SUPPLYREQDATE>'+CONVERT(VARCHAR(20),A.SUPPLYREQDATE)+'</SUPPLYREQDATE>'\n                +'</Key>',\n                1 AS Changed,\n                1 AS CIS_ID,\n                A.LOADCONTROLCREATEID,\n                A.LOADCONTROLCREATEID\n            ",
 '    ICMSTAGE_PEACETX.PM_E_CONSPREM_TB A\n            LEFT    JOIN ICMTRANS_PEACETX.vw_MasterContractLookup B\n                ON  A.[DEBTORNUM] = B.[DEBTORNUM]\n                AND A.[PREMNUM] = B.[PREMNUM]\n                AND A.SUPPLYREQDATE = B.SUPPLYREQDATE \n            WHERE   B.[DEBTORNUM]     

In [113]:
                        #Keeping the first part of stmt_part[0] and appending rest of the array strings in stmt_part[1]
                        for i in range(2,len(stmt_part)):
                             stmt_part[1]=stmt_part[1]+' ƒ '+stmt_part[i]
                        #Replacing special character to from in first part of the split
                        stmt_part_1 = re.sub(r'ƒ', 'from', re.sub(r'‹', r'(', stmt_part[0], flags=re.S|re.I), flags=re.S|re.I)
                        #Replacing special character to from in second part of the split
                        stmt_part_2 = re.sub(r'ƒ', 'from', re.sub(r'‹', r'(', stmt_part[1], flags=re.S|re.I), flags=re.S|re.I)
                        

"INSERT INTO [LOOKUP].[MASTERCONTRACT] (\n                [Key],\n                [Changed],\n                [CIS_ID],\n                LOADCONTROLCREATEID,\n                LOADCONTROLUPDATEID)\n            SELECT  DISTINCT \n                '<Key>'\n                    + '<DEBTORNUM>'+CONVERT(VARCHAR(20),A.DEBTORNUM)+'</DEBTORNUM>'\n                    +'<PREMNUM>'+CONVERT(VARCHAR(20),A.PREMNUM)+'</PREMNUM>'\n                    +'<SUPPLYREQDATE>'+CONVERT(VARCHAR(20),A.SUPPLYREQDATE)+'</SUPPLYREQDATE>'\n                +'</Key>',\n                1 AS Changed,\n                1 AS CIS_ID,\n                A.LOADCONTROLCREATEID,\n                A.LOADCONTROLCREATEID\n            "

In [126]:
print(stmt_part_1)

INSERT INTO [LOOKUP].[MASTERCONTRACT] (
                [Key],
                [Changed],
                [CIS_ID],
                LOADCONTROLCREATEID,
                LOADCONTROLUPDATEID)
            SELECT  DISTINCT 
                '<Key>'
                    + '<DEBTORNUM>'+CONVERT(VARCHAR(20),A.DEBTORNUM)+'</DEBTORNUM>'
                    +'<PREMNUM>'+CONVERT(VARCHAR(20),A.PREMNUM)+'</PREMNUM>'
                    +'<SUPPLYREQDATE>'+CONVERT(VARCHAR(20),A.SUPPLYREQDATE)+'</SUPPLYREQDATE>'
                +'</Key>',
                1 AS Changed,
                1 AS CIS_ID,
                A.LOADCONTROLCREATEID,
                A.LOADCONTROLCREATEID
            


In [114]:
stmt_part_2

'    ICMSTAGE_PEACETX.PM_E_CONSPREM_TB A\n            LEFT    JOIN ICMTRANS_PEACETX.vw_MasterContractLookup B\n                ON  A.[DEBTORNUM] = B.[DEBTORNUM]\n                AND A.[PREMNUM] = B.[PREMNUM]\n                AND A.SUPPLYREQDATE = B.SUPPLYREQDATE \n            WHERE   B.[DEBTORNUM]       IS NULL;'

In [116]:
dest_tbl

'[LOOKUP].[MASTERCONTRACT]'

In [270]:
result=re.findall(r'@\w+.(\w+)','abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')
print(result)

['com', 'in', 'com', 'biz']


In [287]:
result=re.search(r'\[\w+\].\[(\w+)\]',dest_tbl,re.S|re.I).group(1)
result

'MASTERCONTRACT'

In [162]:
col_list_str = re.search(dest_tbl+r'\s.*\w.*', stmt_part_1, re.S|re.I).group(1)
col_list_str

AttributeError: 'NoneType' object has no attribute 'group'

In [163]:
col_list_str = re.search( r'\s*\((.*?)\)[\(\s]*\bSELECT\b', stmt_part_1, re.S|re.I)
col_list_str

<re.Match object; span=(37, 208), match=' (\n                [Key],\n                [Chan>

In [165]:
col_list_str = re.search( r'\s*(\w.*?)\s.*\bSELECT\b', stmt_part_1, re.S|re.I)
col_list_str

<re.Match object; span=(0, 208), match='INSERT INTO [LOOKUP].[MASTERCONTRACT] (\n        >

In [174]:
tbl_alias = re.search(r'\b'+dest_tbl+r'\b\s.*(.*?)\s+(?=SELECT)', stmt_part_1, re.S|re.I).group(1).strip()

AttributeError: 'NoneType' object has no attribute 'group'

In [281]:
upd_set = re.search(r'\b(?<=[MASTERCONTRACT])\s.*\((\w.*)?\s.*(?=SELECT)', stmt_part_1, re.S|re.I).group(1).strip()
print(upd_set)

AttributeError: 'NoneType' object has no attribute 'strip'

In [328]:
upd_table_alias = re.search(r'(?<=MASTERCONTRACT)\s.*\w.*(?=SELECT)', stmt_part_1, re.S|re.I).group().strip()
print(upd_table_alias)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
col_list = newSplit(col_list_str, ',')